In [63]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing

from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss


%matplotlib inline

In [4]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

'wget' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


In [6]:
df = pd.read_csv('loan_train.csv')
df

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male
...,...,...,...,...,...,...,...,...,...,...
341,394,394,COLLECTION,800,15,9/11/2016,9/25/2016,32,High School or Below,male
342,395,395,COLLECTION,1000,30,9/11/2016,10/10/2016,25,High School or Below,male
343,397,397,COLLECTION,800,15,9/12/2016,9/26/2016,39,college,male
344,398,398,COLLECTION,1000,30,9/12/2016,11/10/2016,28,college,male


In [7]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


In [8]:
df['loan_status'].value_counts()

PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64

In [17]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,0
1,2,2,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,1
2,3,3,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,0
3,4,4,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,1
4,6,6,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,0


In [18]:
Feature = df[['Principal','terms','age','Gender']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

,Principal,terms,age,Gender,Bechalor,High School or Below,college
0,1000,30,45,0,0,1,0
1,1000,30,33,1,1,0,0
2,1000,15,27,0,0,0,1
3,1000,30,28,1,0,0,1
4,1000,30,29,0,0,0,1


In [22]:
X = Feature
y = df['loan_status']

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([('scaler', StandardScaler()),('KNN', KNeighborsClassifier())])
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('KNN', KNeighborsClassifier())])

In [36]:
from sklearn.model_selection import GridSearchCV

parameteres = {'KNN__n_neighbors' : [3,5,7,9,11,15]}

grid = GridSearchCV(pipeline, param_grid=parameteres,cv=5)

In [37]:
%%time
grid.fit(X_train,y_train)

Wall time: 237 ms


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('KNN', KNeighborsClassifier())]),
             param_grid={'KNN__n_neighbors': [3, 5, 7, 9, 11, 15]})

In [38]:
grid.best_params_

{'KNN__n_neighbors': 15}

In [68]:
y_pred = grid.predict(X_test)

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, grid.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))

KNNJaccard = jaccard_score(y_test, y_pred, pos_label = 'PAIDOFF')
KNNF1 = f1_score(y_test, y_pred, average='weighted')
print("Avg F1-score: %.2f" % KNNF1 )
print("KNN Jaccard Score: %.2f" % KNNJaccard)

Train set Accuracy:  0.7427536231884058
Test set Accuracy:  0.7857142857142857
Avg F1-score: 0.69
KNN Jaccard Score: 0.79


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([('scaler', StandardScaler()),('DT', DecisionTreeClassifier())])
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('DT', DecisionTreeClassifier())])

In [43]:
parameteres = {}
grid = GridSearchCV(pipeline, param_grid=parameteres,cv=5)

In [45]:
%%time
grid.fit(X_train,y_train)

Wall time: 44.3 ms


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('DT', DecisionTreeClassifier())]),
             param_grid={})

In [76]:
y_pred = grid.predict(X_test)

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, grid.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))

DTJaccard = jaccard_score(y_test, y_pred, pos_label = 'PAIDOFF')
DTF1 = f1_score(y_test, y_pred, average='weighted')
print("Avg F1-score: %.2f" % KNNF1 )
print("KNN Jaccard Score: %.2f" % KNNJaccard)

Train set Accuracy:  0.7427536231884058
Test set Accuracy:  0.7857142857142857
Avg F1-score: 0.69
KNN Jaccard Score: 0.79


In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([('scaler', StandardScaler()),('SVM', SVC())])
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('SVM', SVC())])

In [51]:
parameteres = {}
grid = GridSearchCV(pipeline, param_grid=parameteres,cv=5)

In [52]:
%%time
grid.fit(X_train,y_train)

Wall time: 200 ms


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('SVM', SVC())]),
             param_grid={})

In [78]:
y_pred = grid.predict(X_test)

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, grid.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))

SVMJaccard = jaccard_score(y_test, y_pred, pos_label = 'PAIDOFF')
SVMF1 = f1_score(y_test, y_pred, average='weighted')
print("Avg F1-score: %.2f" % KNNF1 )
print("KNN Jaccard Score: %.2f" % KNNJaccard)

Train set Accuracy:  0.7427536231884058
Test set Accuracy:  0.7857142857142857
Avg F1-score: 0.69
KNN Jaccard Score: 0.79


In [79]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([('scaler', StandardScaler()),('LR', LogisticRegression())])
pipeline

parameteres = {}
grid = GridSearchCV(pipeline, param_grid=parameteres,cv=5)

grid.fit(X_train,y_train)

y_pred = grid.predict(X_test)

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, grid.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_pred))

LRJaccard = jaccard_score(y_test, y_pred, pos_label = 'PAIDOFF')
LRF1 = f1_score(y_test, y_pred, average='weighted')
y_pred = grid.predict_proba(X_test)
Logloss = log_loss(y_test, y_pred)
print("Avg F1-score: %.2f" % KNNF1 )
print("KNN Jaccard Score: %.2f" % KNNJaccard)

Train set Accuracy:  0.7427536231884058
Test set Accuracy:  0.7857142857142857
Avg F1-score: 0.69
KNN Jaccard Score: 0.79


In [83]:
column = {'Algorithm':['knn','dt','svm','lr'],'Jaccard':[KNNJaccard,DTJaccard,SVMJaccard,LRJaccard],'F1-score':[KNNF1,DTF1,SVMF1,LRF1],'LogLoss':[np.nan,np.nan,np.nan,Logloss]}

In [85]:
df2 = pd.DataFrame(columns=column)
df2

,Algorithm,Jaccard,F1-score,LogLoss
